In [23]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Path to your input mp4 file
input_path = "rawfootage_mary(Interview Original).mp4"
output_srt = "output_v2.srt"

import whisper
model = whisper.load_model("base") # or "small", "medium", "large"
result = model.transcribe(input_path, verbose=True)

Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: English
[00:00.000 --> 00:03.080]  Okay, ready to get started?
[00:03.080 --> 00:03.920]  Okay.
[00:03.920 --> 00:06.600]  So start out and just can you introduce yourself,
[00:06.600 --> 00:10.080]  so say your first name and how long you have been living
[00:10.080 --> 00:12.560]  at sunrise of Alexandria.
[00:12.560 --> 00:16.120]  Okay, my name is Mary Sikoski.
[00:16.120 --> 00:19.200]  I've been living here at sunrise almost five months.
[00:19.200 --> 00:22.480]  It'll be the five months on the 24th of this month.
[00:23.640 --> 00:26.120]  Can you do it one more time and just say your first name
[00:26.120 --> 00:27.120]  or last name?
[00:28.120 --> 00:31.440]  My name is Mary Sikoski.
[00:31.440 --> 00:36.440]  I am living here at sunrise happily for almost five months.
[00:39.640 --> 00:43.080]  It will be five months on the 24th of this month.
[00:45.080 --> 00:4

In [3]:
# Extract audio (needed for diarization)
import subprocess
from pathlib import Path

audio_path = Path("temp_audio.wav")

if not audio_path.exists():
    subprocess.run([
        "ffmpeg", "-y", "-i", input_path,
        "-vn",                # no video
        "-ac", "1",           # mono is fine for diarization
        "-ar", "16000",       # 16 kHz sample rate
        "-f", "wav",
        str(audio_path)
    ], check=True)

print("✅ Extracted audio to", audio_path)


✅ Extracted audio to temp_audio.wav


In [ ]:
# Diarization
# Baselining
#  Input video : 21:08 
#   CPU: Interrupted after 8 min
#   GPU: 22.1s lol

DIARIZATION_APPROACH = "GPU"  # "CPU" or "GPU" or "LLM"

# (1) Load diarization pipeline
from pyannote.audio import Pipeline
hf_token = os.getenv("HF_TOKEN")  # needs Hugging Face token for model download


if DIARIZATION_APPROACH == "GPU":
    import torch
    device = torch.device("cuda")
    diar_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=hf_token).to(device) # requires happy CUDA drivers
elif DIARIZATION_APPROACH == "CPU":
    diar_pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization-3.1", use_auth_token=hf_token)

# (2) Run diarization directly on the audio
diarization = diar_pipeline({"audio": audio_path})

# (3) Inspect
for turn, _, speaker in diarization.itertracks(yield_label=True):
    print(f"{turn.start:.0001f}s – {turn.end:.0001f}s: speaker {speaker}")

/home/rari/Documents/video-auto-cut/.venv/lib/python3.12/site-packages/pyannote/audio/core/io.py:212: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  torchaudio.list_audio_backends()
/home/rari/Documents/video-auto-cut/.venv/lib/python3.12/site-packages/speechbrain/utils/torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch

1.4s – 3.1s: speaker SPEAKER_01
3.1s – 3.1s: speaker SPEAKER_00
3.1s – 3.2s: speaker SPEAKER_01
3.2s – 3.2s: speaker SPEAKER_00
3.2s – 3.3s: speaker SPEAKER_01
3.3s – 3.3s: speaker SPEAKER_00
3.3s – 12.3s: speaker SPEAKER_01
12.5s – 13.0s: speaker SPEAKER_00
13.6s – 15.5s: speaker SPEAKER_00
16.1s – 22.3s: speaker SPEAKER_00
23.6s – 27.0s: speaker SPEAKER_01
28.4s – 31.4s: speaker SPEAKER_00
32.7s – 35.0s: speaker SPEAKER_00
35.6s – 36.3s: speaker SPEAKER_00
37.1s – 39.3s: speaker SPEAKER_00
39.8s – 43.1s: speaker SPEAKER_00
45.2s – 45.9s: speaker SPEAKER_01
46.2s – 46.7s: speaker SPEAKER_00
46.7s – 60.7s: speaker SPEAKER_01
47.6s – 48.2s: speaker SPEAKER_00
50.5s – 52.3s: speaker SPEAKER_00
62.1s – 68.0s: speaker SPEAKER_00
68.5s – 72.2s: speaker SPEAKER_00
73.0s – 76.3s: speaker SPEAKER_00
77.5s – 79.0s: speaker SPEAKER_00
79.4s – 80.3s: speaker SPEAKER_00
80.9s – 81.8s: speaker SPEAKER_00
82.2s – 83.1s: speaker SPEAKER_00
83.7s – 85.0s: speaker SPEAKER_00
85.5s – 86.3s: speaker SPEA

In [19]:
import pandas as pd

df_w = pd.DataFrame(result["segments"])[["start", "end", "text"]]

PAD = 0.2
df_d = pd.DataFrame(
    [(turn.start - PAD, turn.end + PAD, spk)
     for turn, _, spk in diarization.itertracks(yield_label=True)],
    columns=["d_start", "d_end", "speaker"]
)
audio_dur = df_w["end"].max()
df_d["d_start"] = df_d["d_start"].clip(lower=0)
df_d["d_end"] = df_d["d_end"].clip(upper=audio_dur)

df_join = (
    df_w.assign(key=1)
    .merge(df_d.assign(key=1), on="key")
    .drop("key", axis=1)
    .query("end > d_start and start < d_end")
)

df_join["overlap"] = (
    df_join[["end", "d_end"]].min(axis=1)
    - df_join[["start", "d_start"]].max(axis=1)
)
df_join = df_join[df_join["overlap"] > 0]

df_aligned = (
    df_join.loc[df_join.groupby(["start", "end", "text"])["overlap"].idxmax()]
    [["start", "end", "text", "speaker"]]
    .sort_values("start")
)

def write_srt(df, filename):
    def fmt(ts):
        h, m, s = int(ts // 3600), int((ts % 3600) // 60), ts % 60
        return f"{h:02d}:{m:02d}:{s:06.3f}".replace(".", ",")

    with open(filename, "w", encoding="utf-8") as f:
        for i, row in enumerate(df.itertuples(index=False), start=1):
            dur = row.end - row.start
            f.write(
                f"{i}\n"
                f"{fmt(row.start)} --> {fmt(row.end)} (duration: {dur:.2f}s)\n"
                f"[{row.speaker}] {row.text.strip()}\n\n"
            )

write_srt(df_aligned, "output_v2.srt")
print(f"✅ SRT saved: {len(df_aligned)} segments written")



✅ SRT saved: 413 segments written


In [ ]:
from pathlib import Path
import pickle

# This shit takes AGES on CPU. Save it.
diar_save = Path("diarization_cpu.pkl")
with diar_save.open("wb") as f:
    pickle.dump(diarization, f)
print(f"✅ Saved diarization result to {diar_save.resolve()}")

# Other thoughts:
# Swithc for CPU vs GPU vs LLM based diarization


In [20]:
from pathlib import Path
import re

def extract_text_from_srt(srt_path: str) -> str:
    text_blocks = []
    with open(srt_path, "r", encoding="utf-8") as f:
        block = []
        for line in f:
            line = line.strip()
            if not line:
                if block:
                    text_blocks.append(" ".join(block))
                    block = []
            else:
                block.append(line)
        if block:
            text_blocks.append(" ".join(block))
    # join each SRT block on a newline so GPT can see structure
    return "\n".join(text_blocks)

srt_text = extract_text_from_srt("output_v2.srt")
print(srt_text[:500])  # sanity check


1 00:00:00,000 --> 00:00:03,080 (duration: 3.08s) [SPEAKER_01] Okay, ready to get started?
2 00:00:03,080 --> 00:00:03,920 (duration: 0.84s) [SPEAKER_01] Okay.
3 00:00:03,920 --> 00:00:06,600 (duration: 2.68s) [SPEAKER_01] So start out and just can you introduce yourself,
4 00:00:06,600 --> 00:00:10,080 (duration: 3.48s) [SPEAKER_01] so say your first name and how long you have been living
5 00:00:10,080 --> 00:00:12,560 (duration: 2.48s) [SPEAKER_01] at sunrise of Alexandria.
6 00:00:12,560 -->


In [ ]:
import os
import json
from openai import OpenAI

API_KEY = os.getenv("OPENAI_API_KEY")

if not API_KEY:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

client = OpenAI(
    api_key = API_KEY
)

max_duration = 120
model_name = "gpt-5"#-mini

# --- build prompt ---
system_prompt = f"""
You are a video editor creating a concise {max_duration}-second story from a full interview transcript in SRT format.
Your task is to down-select the most meaningful spoken moments that together form a cohesive story.

Return only the essential timestamped dialogue segments that preserve:
- chronological order
- emotional continuity and completeness of thought
- total combined runtime of about {max_duration} seconds (±10 seconds)
- ie Select segments totaling close to 120 seconds of cumulative duration
- use timestamps exactly as in the transcript
- do not fabricate or merge lines from different timestamps
- ignore visual or b-roll info
- preserve tone and authenticity
- Remove any segments that are interviewer or narrator questions.
- If you merge adjacent dialogue lines, set the "start" equal to the earliest start time among them,
  and the "end" equal to the latest end time among them.

Output only valid JSON: a list of objects like
[
  {{"start": "HH:MM:SS.xx", "end": "HH:MM:SS.xx", "spoken_text": "verbatim text"}}
]
"""

user_prompt = f"Here is the full SRT transcript:\n\n{srt_text}\n\nReturn only the JSON list."

resp = client.responses.create(
    model="gpt-5",
    input=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ],
    reasoning={"effort": "low"},
)

In [34]:
raw = resp.output_text.strip()

# --- clean fences if present ---
if "```" in raw:
    raw = re.sub(r"^```[a-zA-Z0-9]*\n?", "", raw)
    raw = raw.replace("```", "").strip()

# --- parse JSON safely ---
try:
    segments = json.loads(raw)
except json.JSONDecodeError:
    Path("story_segments_raw.txt").write_text(raw, encoding="utf-8")
    raise RuntimeError("Model output was not valid JSON; raw text saved to story_segments_raw.txt")

def normalize_ts(ts: str) -> str:
    """Convert SRT-style timestamps (00:00:12,560) -> ffmpeg-friendly (00:00:12.560)."""
    return re.sub(r",", ".", ts.strip())

for seg in segments:
    seg["start"] = normalize_ts(seg["start"])
    seg["end"]   = normalize_ts(seg["end"])

# --- save clean JSON ---
Path("story_segments.json").write_text(json.dumps(segments, indent=2), encoding="utf-8")
print(f"✅ Saved {len(segments)} cleaned segments to story_segments.json")


✅ Saved 4 cleaned segments to story_segments.json


In [35]:
import json, subprocess
from pathlib import Path

input_video = "rawfootage_mary(Interview Original).mp4"
segments_file = "story_segments.json"
output_dir = Path("trimmed_clips")
output_dir.mkdir(exist_ok=True)

segments = json.loads(Path(segments_file).read_text(encoding="utf-8"))

def ffmpeg_trim(input_path, start, end, output_path):
    """Trim reliably, allowing sub-second accuracy."""
    cmd = [
        "ffmpeg",
        "-hide_banner",
        "-loglevel", "error",      # quieter logs
        "-ss", start,
        "-to", end,
        "-i", input_path,
        "-c:v", "libx264",         # re-encode ensures playable output
        "-c:a", "aac",
        "-strict", "experimental",
        "-y",
        str(output_path)
    ]
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"❌ ffmpeg failed for {output_path.name}")
        print(result.stderr)
    else:
        print(f"✅ {output_path.name}")

for i, seg in enumerate(segments, start=1):
    start, end = seg["start"], seg["end"]
    clip_name = f"clip_{i:02d}_{start.replace(':', '-')}_{end.replace(':', '-')}.mp4"
    out_path = output_dir / clip_name
    ffmpeg_trim(input_video, start, end, out_path)

print(f"\nAll done. Check clips in {output_dir.resolve()}")


✅ clip_01_00-00-31.440_00-00-36.440.mp4
✅ clip_02_00-01-08.560_00-02-37.120.mp4
✅ clip_03_00-11-01.760_00-11-17.840.mp4
✅ clip_04_00-11-57.280_00-12-13.760.mp4

All done. Check clips in /home/rari/Documents/video-auto-cut/trimmed_clips


In [36]:
import subprocess
from pathlib import Path

# --- config ---
clips_dir = Path("trimmed_clips")
output_video = "story_concat.mp4"

# --- collect clips in natural order ---
clip_files = sorted(clips_dir.glob("*.mp4"))

# --- create ffmpeg concat list file ---
concat_file = Path("concat_list.txt")
with concat_file.open("w", encoding="utf-8") as f:
    for clip in clip_files:
        # ffmpeg concat demuxer requires "file 'path'"
        f.write(f"file '{clip.resolve()}'\n")

# --- concat using ffmpeg demuxer ---
cmd = [
    "ffmpeg",
    "-y",
    "-f", "concat",
    "-safe", "0",
    "-i", str(concat_file),
    "-c", "copy",
    output_video
]

subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print(f"✅ Concatenated {len(clip_files)} clips into {output_video}")


✅ Concatenated 4 clips into story_concat.mp4
